In [1]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('Model/CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Model/CNN_model_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [2]:
import numpy
print(numpy.__version__)

1.19.5


In [10]:
import pickle

with open('Model/scaler2.pickle', 'rb') as f:
    scaler2 = pickle.load(f)
    
with open('Model/encoder2.pickle', 'rb') as f:
    encoder2 = pickle.load(f)

    
print("Done")   

Done


In [11]:
import librosa
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])
    
    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [12]:
import numpy as np
import librosa
def get_predict_feat(path):
    d, s_rate= librosa.load(path, duration=2.5, offset=0.6)
    res=extract_features(d)
    features=np.array(res)
    result= np.zeros(2376)
    result[:features.size] = features
    result=np.reshape(result,newshape=(1,2376))
    i_result = scaler2.transform(result)
    final_result=np.expand_dims(i_result, axis=2)
    
    return final_result

In [13]:
emotions1={1:'Neutral', 2:'Calm', 3:'Happy', 4:'Sad', 5:'Angry', 6:'Fear', 7:'Disgust',8:'Surprise'}
def prediction(path1):
    res=get_predict_feat(path1)
    predictions=loaded_model.predict(res)
    y_pred = encoder2.inverse_transform(predictions)
    print(y_pred[0][0])  
    result = y_pred[0][0]
    return result

In [14]:
import sys
import warnings
import librosa
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
result = prediction("prueba_178.150-195.000.wav")
result.type()

angry


AttributeError: 'str' object has no attribute 'type'

In [15]:
res=get_predict_feat("DATA/DATA/AudioWAV/1001_DFA_ANG_XX.wav")
print(res.shape)

(1, 2376, 1)


In [16]:
prediction("DATA/DATA/AudioWAV/1001_DFA_ANG_XX.wav")

angry


'angry'

In [17]:
import os
Crema = "DATA/DATA/AudioWAV/"
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []
good = 0
bad = 0

for file in crema_directory_list:
    result = prediction(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        emotion = "sad"
    elif part[2] == 'ANG':
        emotion = "angry"
    elif part[2] == 'DIS':
        emotion = "disgust"
    elif part[2] == 'FEA':
        emotion = "fear"
    elif part[2] == 'HAP':
        emotion = "happy"
    elif part[2] == 'NEU':
        emotion = "neutral"
    else:
        emotion = "Unknown"
        
    if result == emotion:
        good += 1
    else:
        bad += 1
        
print(good, bad)

fear
disgust
disgust
fear
fear
happy
neutral
neutral
fear
disgust
neutral
fear
angry
angry
angry
happy
fear
disgust
sad
happy
fear
sad
fear
fear
happy
neutral
happy
fear
sad
fear
happy
disgust
angry
angry
disgust
disgust
sad
disgust
disgust
sad
neutral
fear
disgust
happy
sad
happy
neutral
happy
disgust
happy
sad
happy
neutral
happy
neutral
sad
fear
angry
angry
happy
angry
neutral
neutral
disgust
neutral
disgust
angry
angry
angry
fear
sad
neutral
angry
neutral
disgust
happy
disgust
happy
happy
angry
sad
angry
sad
neutral
neutral
angry
fear
fear
sad
angry
neutral
disgust
disgust
disgust
happy
happy
disgust
fear
disgust
disgust
disgust
sad
fear
happy
disgust
happy
fear
sad
disgust
angry
angry
disgust
happy
fear
angry
angry
angry
angry
sad
fear
fear
happy
sad
disgust
happy
angry
angry
sad
disgust
fear
fear
fear
disgust
sad
neutral
happy
happy
fear
fear
disgust
happy
fear
sad
sad
fear
disgust
fear
angry
fear
sad
sad
neutral
fear
happy
neutral
happy
happy
sad
fear
sad
sad
happy
neutral
happy